In [ ]:
options validvarname=any;


[<class 'saspy.sasexceptions.SASIOConnectionError'>, SASIOConnectionError('No SAS process attached. SAS process has terminated unexpectedly.'), <traceback object at 0x7f82a8bda640>]


SAS Connection failed. No connection established. Double check your settings in sascfg_personal.py file.
Attempted to run program /opt/sasinside/SASHome/SASFoundation/9.4/bin/sas_u8 with the following parameters:['/opt/sasinside/SASHome/SASFoundation/9.4/bin/sas_u8', '-nodms', '-stdio', '-terminal', '-nosyntaxcheck', '-pagesize', 'MAX', '']
Try running the following command (where saspy is running) manually to see if you can get more information on what went wrong:
/opt/sasinside/SASHome/SASFoundation/9.4/bin/sas_u8 -nodms -stdio -terminal -nosyntaxcheck -pagesize MAX  

No SAS process attached. SAS process has terminated unexpectedly.
Invalid response from SAS on inital submission. printing the SASLOG as diagnostic



## Start
Pull data from census.gov : https://www.census.gov/data/developers/data-sets/economic-indicators.html

In [ ]:
filename resjson temp;

proc http
    url='https://api.census.gov/data/timeseries/eits/resconst?get=cell_value,data_type_code,time_slot_id,error_data,category_code,geo_level_code,seasonally_adj,time_slot_name&time=from+2010'
    method='GET'
    out=resjson;
run;

Assign a JSON library to the HTTP response

In [ ]:
libname res_data JSON fileref=resjson;

data temp;
  set res_data.root;
where element1 ^= 'cell_value';
run;


Transform some of the data items

In [ ]:

data housing;
  set temp (RENAME= (element2=data_type_code element3=time_slot_id element4=error_data
element5=category_code element6=geo_level_code element7=seasonally_adj element8=time_slot_name element9=time)
);
length cell_value 8.;
cell_value = input(element1,comma9.);
run;

What does the data look like?

In [ ]:
proc contents data=housing; run;

Copy into cas

In [ ]:
cas;
caslib _all_ assign;

proc casutil incaslib="Public" outcaslib="Public";

    droptable casdata="housing" quiet;
    load data=work.housing casout="housing" promote;
    save casdata="housing" casout="housing" replace;
run;